In [25]:
import os
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
from sklearn.model_selection import train_test_split

In [26]:
PATH_CSV_LABELS = '../tabellaPseudoLabelsDatasetCompleto.csv'
PATH_DATASET_LAVORATO = '../../Nuovi Dati/TemplateGeneratiClean'
PATH_MODELLO = '../../Modelli-v2/modello_pesi_inizializzati_gaussian_blur'
PATH_MODELLO_FINALE_RIADDESTRATO = '../../Modelli-v2/modello_finale_riaddestrato_gaussian_blur'

N_IMG = 1
FORMATO_IMG = 'jpg'
CHANNELS = 1
HEIGHT = 512
WIDTH = 512
TARGET_SIZE = (WIDTH, HEIGHT)

HUBER_DELTA = 1.0

In [27]:
df_labels = pd.read_csv(PATH_CSV_LABELS)
df_labels.head(5)

,Acquisizione,Label,Label_norm
0,AgatielloRoberto_000,0.648876,50.428711
1,AgatielloRoberto_001,0.759503,61.300294
2,AgatielloRoberto_002,0.747500,60.120716
3,AgatielloRoberto_003,0.818899,67.137182
4,AgatielloRoberto_004,0.825895,67.824714


In [28]:
X = []
y = []

for id_acquisizione in df_labels.Acquisizione:
    path_img = os.path.join(PATH_DATASET_LAVORATO, id_acquisizione, 'Immagini', 'immagine1.jpg')
    image = Image.open(path_img)
    resized_image = image.resize(TARGET_SIZE)
    image_array = np.array(resized_image)
    normalized_image_array = image_array / 255.0
    
    pseudo_label = df_labels[df_labels.Acquisizione == id_acquisizione].Label_norm
    
    X.append(normalized_image_array)
    y.append(pseudo_label)
    
X = np.array(X)
y = np.array(y)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=43)

In [30]:
print(f'X_train: {len(X_train)}')
print(f'X_test: {len(X_test)}')

X_train: 574
X_test: 144


In [31]:
model = keras.models.load_model(PATH_MODELLO)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 510, 510, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 255, 255, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 2080800)           0         
                                                                 
 dense (Dense)               (None, 64)                133171264 
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 133,171,649
Trainable params: 133,171,649
Non-trainable params: 0
__________________________________________

In [32]:
# model.fit(x=X_train, y=y_train, batch_size=1, epochs=20, validation_data=(X_cv, y_cv))

In [33]:
#model.predict(X)

In [34]:
#y

In [35]:
# model.save(PATH_MODELLO_FINALE)

## Addestramento nuovo modello con Huber loss

In [36]:
conv1_weights = model.layers[0].weights
dense1_weights = model.layers[3].weights
dense2_weights = model.layers[4].weights

new_model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(HEIGHT, WIDTH, CHANNELS), weights=conv1_weights),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu', weights=dense1_weights),
    layers.Dense(1, weights=dense2_weights)
])

In [37]:
new_model.compile(optimizer='adam', loss=tf.keras.losses.Huber(delta=HUBER_DELTA), metrics=['mae'])

In [38]:
new_model.fit(x=X_train, y=y_train, batch_size=1, epochs=10)

Epoch 1/10


2023-07-26 12:35:04.861087: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


574/574 [==============================] - 67s 116ms/step - loss: 16.7362 - mae: 17.2308
Epoch 2/10
574/574 [==============================] - 66s 115ms/step - loss: 16.1623 - mae: 16.6552
Epoch 3/10
574/574 [==============================] - 68s 118ms/step - loss: 15.4924 - mae: 15.9845
Epoch 4/10
574/574 [==============================] - 67s 117ms/step - loss: 12.9821 - mae: 13.4763
Epoch 5/10
574/574 [==============================] - 67s 117ms/step - loss: 11.1972 - mae: 11.6885
Epoch 6/10
574/574 [==============================] - 66s 115ms/step - loss: 8.3288 - mae: 8.8172
Epoch 7/10
574/574 [==============================] - 66s 115ms/step - loss: 6.1992 - mae: 6.6842
Epoch 8/10
574/574 [==============================] - 66s 115ms/step - loss: 5.0259 - mae: 5.5056
Epoch 9/10
574/574 [==============================] - 66s 115ms/step - loss: 4.0403 - mae: 4.5122
Epoch 10/10
574/574 [==============================] - 66s 115ms/step - loss: 3.6484 - mae: 4.1216


In [39]:
#new_model.predict(X)

In [40]:
#y

In [41]:
new_model.save(PATH_MODELLO_FINALE_RIADDESTRATO)

INFO:tensorflow:Assets written to: ../../Modelli-v2/modello_finale_riaddestrato_gaussian_blur/assets


INFO:tensorflow:Assets written to: ../../Modelli-v2/modello_finale_riaddestrato_gaussian_blur/assets


### Import nuovo modello

In [21]:
new_model = tf.keras.models.load_model(PATH_MODELLO_FINALE_RIADDESTRATO)

In [22]:
def avg(lst):
    return sum(lst) / len(lst)

In [24]:
avg(abs(new_model.predict(X) - y))

23/23 [==============================] - 5s 227ms/step


array([5.7384091])